In [1]:
!pip install git+https://github.com/ourownstory/neural_prophet.git --upgrade

  Cloning https://github.com/ourownstory/neural_prophet.git to /private/var/folders/t6/zc4n8b2j6rdcj9p7xb7gxm0h0000gn/T/pip-req-build-dp9oh2q1
  Running command git clone --filter=blob:none --quiet https://github.com/ourownstory/neural_prophet.git /private/var/folders/t6/zc4n8b2j6rdcj9p7xb7gxm0h0000gn/T/pip-req-build-dp9oh2q1
  Resolved https://github.com/ourownstory/neural_prophet.git to commit de24e5d041f4a9d5b2b262d7e530b8243d9a8b1f
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached holidays-0.14.2-py3-none-any.whl (179 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 2.1 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 15.3 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 MB 14.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached c

In [2]:
import pickle
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level


In [17]:


target_column = 'Total Vertical TBS'

df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns = {target_column:'y', 'Adm. requests cum':"Adm requests cum", 'Pts.waiting for admission CUM':'Pts waiting for admission CUM'}, inplace = True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

m = NeuralProphet(
  # growth='off',
  # yearly_seasonality=False,
  # weekly_seasonality=True,
  # daily_seasonality=True,
  n_lags=48,
  n_forecasts=12,
  # changepoints_range=0.95,
  # n_changepoints=50,
  # num_hidden_layers=4,
  # d_hidden=36,
  # learning_rate=0.005,
)
m = m.add_lagged_regressor(names=regressors)
m = m.add_country_holidays("CA")
metrics = m.fit(df, 
                freq='H', 
                # progress='plot'
                )
print(metrics.tail(1))

with open('../models/verticalTBS_forecast_model.pkl', "wb") as f:
    pickle.dump(m, f)

INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', number of lags for regressor is set to Autoregression number of lags (48)
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.439% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.forecaster.__handle_missing_data) - 63 missing dates added.
WARNING - (NP.forecaster.__handle_missing_data) - 63 missing values in column y were detected in total. 
INFO - (NP.forecaster.__handle_missing_data) - 63 NaN values in column y were auto-imputed.
WARNING - (NP.forecaster.__handle_missing_data) - 63 missing values in column Stretcher Pts hrly were detected in total. 
INFO - (NP.forecaster.__handle_missing_data) - 63 NaN values in column Stretcher Pts hrly were auto-imputed.
WARNING - (NP.forecaster.__handle_missing_data) - 63 missing values in column Stretcher Pts cum were detected in total. 
INFO - (NP.forecaster.__handle_missing_data) - 63 NaN value

  0%|          | 0/149 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 7.80E-04, min: 2.58E-02


  0%|          | 0/149 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 7.80E-04, min: 4.52E-02
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 3.93E-03
Epoch[105/105]: 100%|██████████| 105/105 [03:33<00:00,  2.03s/it, SmoothL1Loss=0.0143, MAE=4.12, RMSE=5.23, Loss=0.0105, RegLoss=0]

     SmoothL1Loss       MAE      RMSE     Loss  RegLoss
104      0.014331  4.121945  5.234521  0.01051      0.0


In [18]:

# loaded_model = pickle.load(open('../models/verticalTBS_forecast_model.pkl', 'rb'))
loaded_model = m


In [20]:

target_column = 'Total Vertical TBS'
df = pd.read_csv('https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns = {target_column:'y', 'Adm. requests cum':"Adm requests cum", 'Pts.waiting for admission CUM':'Pts waiting for admission CUM'}, inplace = True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')
# df['ID']='test'


In [21]:
df.head()

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,Total Inflow hrly,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,GARAGE patient TBS,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,y
9097,9,108,5,142,14,250,6,39,0,27,...,1.0,1,0,0,3,0,0,2021-07-26 19:00:00,10,18.0
9096,5,113,6,148,11,261,2,41,0,33,...,1.0,1,0,0,3,1,0,2021-07-26 20:00:00,12,9.0
9095,5,118,2,150,7,268,1,42,0,34,...,0.0,1,0,0,4,4,0,2021-07-26 21:00:00,8,4.0
9094,5,123,8,158,13,281,2,44,0,34,...,0.0,1,0,0,3,5,0,2021-07-26 22:00:00,9,6.0
9093,3,126,7,165,10,291,3,47,0,37,...,1.0,0,0,1,3,6,0,2021-07-26 23:00:00,10,11.0


In [22]:

future = loaded_model.make_future_dataframe(df, periods=12) # periods=m.n_forecasts, n_historic_predictions=False


INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.439% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [23]:
future.tail()

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,Total Inflow hrly,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,GARAGE patient TBS,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,y
55,None,None,None,None,None,None,None,None,None,None,...,NaN,None,None,None,None,None,None,2022-08-12 19:00:00,None,NaN
56,None,None,None,None,None,None,None,None,None,None,...,NaN,None,None,None,None,None,None,2022-08-12 20:00:00,None,NaN
57,None,None,None,None,None,None,None,None,None,None,...,NaN,None,None,None,None,None,None,2022-08-12 21:00:00,None,NaN
58,None,None,None,None,None,None,None,None,None,None,...,NaN,None,None,None,None,None,None,2022-08-12 22:00:00,None,NaN
59,None,None,None,None,None,None,None,None,None,None,...,NaN,None,None,None,None,None,None,2022-08-12 23:00:00,None,NaN


In [37]:

forecast = loaded_model.predict(
    future, 
    # raw=True, 
    decompose=False
    )
# forecast = loaded_model.predict(future)
forecast


INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 98.333% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 98.333% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


,ds,y,residual1,yhat1,residual2,yhat2,residual3,yhat3,residual4,yhat4,...,residual8,yhat8,residual9,yhat9,residual10,yhat10,residual11,yhat11,residual12,yhat12
0,2022-08-10 12:00:00,24.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
1,2022-08-10 13:00:00,26.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
2,2022-08-10 14:00:00,31.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
3,2022-08-10 15:00:00,23.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
4,2022-08-10 16:00:00,27.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
5,2022-08-10 17:00:00,19.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
6,2022-08-10 18:00:00,18.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
7,2022-08-10 19:00:00,17.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
8,2022-08-10 20:00:00,20.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
9,2022-08-10 21:00:00,19.0,NaN,None,NaN,None,NaN,None,NaN,None,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None


In [67]:
forecast_trimmed = forecast[forecast.y.isnull()].set_index('ds')

In [68]:

start = forecast_trimmed.index.tolist()[0]
forecast_length = len(forecast_trimmed.index.tolist())

In [69]:
i = 0
timestamp = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output_list = []
for ds in pd.date_range(start=start, periods=forecast_length, freq='H'):
    i = i+1
    forecast_output_list.append({'ds':ds, 'verticalTBS':forecast_trimmed.loc[ds]['yhat'+str(i)], 'timestamp':timestamp})
forecast_output = pd.DataFrame(forecast_output_list)
forecast_output

,ds,verticalTBS,timestamp
0,2022-08-12 12:00:00,19.287329,2022-08-12 12:14:27
1,2022-08-12 13:00:00,18.646053,2022-08-12 12:14:27
2,2022-08-12 14:00:00,21.020023,2022-08-12 12:14:27
3,2022-08-12 15:00:00,19.545578,2022-08-12 12:14:27
4,2022-08-12 16:00:00,22.569710,2022-08-12 12:14:27
5,2022-08-12 17:00:00,17.739473,2022-08-12 12:14:27
6,2022-08-12 18:00:00,12.435977,2022-08-12 12:14:27
7,2022-08-12 19:00:00,9.325451,2022-08-12 12:14:27
8,2022-08-12 20:00:00,10.364691,2022-08-12 12:14:27
9,2022-08-12 21:00:00,10.015965,2022-08-12 12:14:27
